### Clean data script
A very quick script to process the data for .  Note that the data are so large and the EPA servers are so inconsistent that even the `requests` module with streaming data fails.  So just download the data to your local machine to process.  The objective is to just get started with some real data for California.

In [24]:
import pandas as pd
import geopandas as gpd

In [ ]:
# Grab and unzip the latest verion of the EJSCREEN data for 2020 in the `/tmp` folder.

# Found here (479MB): 
# https://gaftp.epa.gov/ejscreen/2020/EJSCREEN_2020_StatePctile.gdb.zip

gdf = gpd.read_file("/tmp/EJSCREEN_2020_StatePctile.gdb")

In [30]:
# Filter and save the results for just California.  It is still quite large, given 
# the boundaries of the Census tracts (approx. 263MB)
cali = gdf[gdf.ST_ABBREV == "CA"]
cali_attr = cali[["ID", "D_PM25_2", "geometry"]].reset_index(drop=True)
cali_attr.to_file("/tmp/cali.geojson", driver="GeoJSON") 